In [1]:
import pandas as pd
import json

In [2]:
dev_db = r'Dataset\Procesado\dev_db.parquet'

In [3]:
df = pd.read_parquet(dev_db)
df.sample(5)

,item_id,malo,neutral,positivo,developer,release_date,price
1115,230410,0,0,1,Digital Extremes,2013-03-25,0.00
14245,236870,0,0,1,"IO Interactive A/S,Feral Interactive (Linux),F...",2016-03-11,0.00
41529,319630,0,0,1,"DONTNOD Entertainment,Feral Interactive (Mac),...",2015-01-29,0.00
38533,207140,0,0,1,DoubleDutch Games,2016-04-19,14.99
19844,214950,0,0,1,Creative Assembly,2013-09-02,59.95


## <div style="color: lightgreen; text-align: center;"> ...  def developer  ... </div>

In [17]:
aggregation_functions = {
    'item_id': 'sum',  # Suma de la columna 'item_id' por año
    'price': lambda x: (x == 0).sum()  # Conteo de filas donde 'price' es igual a 0 por año
}

In [18]:
resultado_por_anio =df.groupby(df['release_date'].dt.year).agg(aggregation_functions)
print(resultado_por_anio)


                 item_id  price
release_date                   
1989              327950      0
1990               69060      0
1991              358260      0
1992             1000510      0
1993              601360      0
1994             2011620      0
1995              631220      0
1996             1915810      0
1997             5525240      0
1998             4066950      0
1999            11282610     29
2000              420490      0
2001             2643970      0
2002             1970420      0
2003            28473190     18
2004             4347200      0
2005             9695220      0
2006            37270850     89
2007            18413007   3805
2008            12956240     15
2009            28213514     32
2010            70917042    187
2011           287267690    550
2012           929093198   1391
2013          1532793964   1064
2014          1622193042   1424
2015          2029546537   1827
2016           941960799    602
2017           646045970   1578


In [22]:
developer = resultado_por_anio.to_json()

'{"numero de items":{"1989":327950,"1990":69060,"1991":358260,"1992":1000510,"1993":601360,"1994":2011620,"1995":631220,"1996":1915810,"1997":5525240,"1998":4066950,"1999":11282610,"2000":420490,"2001":2643970,"2002":1970420,"2003":28473190,"2004":4347200,"2005":9695220,"2006":37270850,"2007":18413007,"2008":12956240,"2009":28213514,"2010":70917042,"2011":287267690,"2012":929093198,"2013":1532793964,"2014":1622193042,"2015":2029546537,"2016":941960799,"2017":646045970},"Contenido Free":{"1989":0,"1990":0,"1991":0,"1992":0,"1993":0,"1994":0,"1995":0,"1996":0,"1997":0,"1998":0,"1999":29,"2000":0,"2001":0,"2002":0,"2003":18,"2004":0,"2005":0,"2006":89,"2007":3805,"2008":15,"2009":32,"2010":187,"2011":550,"2012":1391,"2013":1064,"2014":1424,"2015":1827,"2016":602,"2017":1578}}'

In [21]:
type(resultado_por_anio)

pandas.core.frame.DataFrame

In [19]:
resultado_por_anio.columns = ['numero de items', 'Contenido Free']

In [20]:
print(resultado_por_anio)

              numero de items  Contenido Free
release_date                                 
1989                   327950               0
1990                    69060               0
1991                   358260               0
1992                  1000510               0
1993                   601360               0
1994                  2011620               0
1995                   631220               0
1996                  1915810               0
1997                  5525240               0
1998                  4066950               0
1999                 11282610              29
2000                   420490               0
2001                  2643970               0
2002                  1970420               0
2003                 28473190              18
2004                  4347200               0
2005                  9695220               0
2006                 37270850              89
2007                 18413007            3805
2008                 12956240     

In [13]:
#Cantidad de items
c_items = df[df['developer'] == 'Valve'].groupby(df['release_date'].dt.year)['item_id'].sum()

In [16]:
type(c_items)

pandas.core.series.Series

In [6]:
df['developer'].value_counts()

developer
Valve                      9557
Facepunch Studios          2550
Bohemia Interactive        1161
Smartly Dressed Games      1066
Re-Logic                    741
                           ... 
Alawar Entertainment          1
Bacon Wrapped Games           1
Qwiboo Ltd                    1
Cliffhanger Productions       1
Aeon Dream Studios            1
Name: count, Length: 2030, dtype: int64

## <div style="color: lightgreen; text-align: center;"> .  def best_developer_year  . </div>
